### GIL锁

熟悉python的都知道，在C语言写的python解释器中存在全局解释器锁，由于全局解释器锁的存在，在同一时间内，python解释器只能运行一个线程的代码，这大大影响了python多线程的性能。而这个解释器锁由于历史原因，现在几乎无法消除。 
  
python GIL 之所以会影响多线程等性能，是因为在多线程的情况下，只有当线程获得了一个全局锁的时候，那么该线程的代码才能运行，而全局锁只有一个，所以使用python多线程，在同一时刻也只有一个线程在运行，因此在即使在多核的情况下也只能发挥出单核的性能。

上述内容引用自链接：https://www.jianshu.com/p/c75ed8a6e9af  


In [35]:
from concurrent.futures import ThreadPoolExecutor

In [183]:
# Worker数量
N = 4

In [184]:
# 线程池
pool = ThreadPoolExecutor(max_workers=N)

### 1. IO密集型操作
一个线程执行IO密集型操作的时候，CPU处于闲置状态，因此GIL会被释放给其他线程，从而缩短了总体的等待运行时间。

In [185]:
from time import sleep, time

In [186]:
def sleep_n_seconds(n=4):
    sleep(n)
    return True

In [187]:
# 串行版本

def serial_func(fn, n_times=N):
    ret = []
    for _ in range(n_times):
        ret.append(fn())
    print(ret)

In [188]:
# 多线程版本，注意要传输lambda函数，sleep时间是准确的

def concurrent_func(fn, n_times=N):
    print(list(pool.map(lambda x: fn(), range(n_times))))
    

In [189]:
def time_it(fn, way):
    assert way in ("serial", "concurrent", "parallel"), "参数way必须是'serial', 'concurrent'或者'parallel'！"
    f = {"serial":serial_func, "concurrent": concurrent_func, "parallel": parallel_func}[way]
    start = time()
    f(fn)
    print("%s版本的运行时间为 %.3f 秒!" % (way, time() - start))

In [190]:
time_it(sleep_n_seconds, "serial")

[True, True, True, True]
serial版本的运行时间为 16.003 秒!


In [191]:
time_it(sleep_n_seconds, "concurrent")

[True, True, True, True]
concurrent版本的运行时间为 4.006 秒!


### 2. CPU密集型操作
一个线程执行CPU密集型操作的时候，CPU处于忙碌状态，运行1000个字节码之后GIL会被释放给其他线程，加上切换线程的时间反而会比串行代码更慢。

In [192]:
def add_one(n_times=10**7):
    ret = 0
    while ret < n_times:
        ret += 1
    return ret

In [193]:
time_it(add_one, "serial")

[10000000, 10000000, 10000000, 10000000]
serial版本的运行时间为 2.301 秒!


In [194]:
time_it(add_one, "concurrent")

[10000000, 10000000, 10000000, 10000000]
concurrent版本的运行时间为 2.716 秒!


### 3. 绕过GIL锁
著名的Cython模块提供了绕过GIL锁的方法，'with nogil'。我们可以用Cython模块重新编写刚才的'add_one'函数，再用python包装为python可调用的函数。代码如下：

In [195]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [196]:
%%cython
def _add_one(long n_times= 10 ** 7):
    cdef long ret = 0
    with nogil:
        while ret < n_times:
            ret += 1
    return ret

def add_one(n_times=10 ** 7):
    return _add_one(n_times)

In [197]:
time_it(add_one, "serial")

[10000000, 10000000, 10000000, 10000000]
serial版本的运行时间为 0.031 秒!


In [198]:
time_it(add_one, "concurrent")

[10000000, 10000000, 10000000, 10000000]
concurrent版本的运行时间为 0.020 秒!
